> 作者：Fire

> 截止日期：2018/6/15

> 截止期数：18068

使用了从03年到截止期数的所有数据，但是也才两千多条，所以结果可能不会太好。肯定会过拟合

### Future work
* 玄学调参
* 构造数据集，每期相差一个红球也可以，这样数据集成6的指数级增长。但是只能用作训练不同模型，因为时间序列是固定的

In [1]:
import keras
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.layers.core import Dense, RepeatVector
from keras.layers import Dropout

#玄学参数 随机种子
win_number = 2018
from numpy.random import seed
seed(win_number)
from tensorflow import set_random_seed
set_random_seed(win_number)    

Using TensorFlow backend.


In [14]:
history = pd.read_csv('./data/ballHisAll.csv')
print(history.shape)
history.head()

(2278, 7)


,red1,red2,red3,red4,red5,red6,blue
0,10,11,12,13,26,28,11
1,4,9,19,20,21,26,12
2,1,7,10,23,28,32,16
3,4,6,7,10,13,25,3
4,4,6,15,17,30,31,16


## 1.红球seq2seq

* 思路：每次通过上一期和当前期建立一个序列对，变成一个长度固定为6的标准seq2seq问题。
* 假设：
    1. 彩票客观/主管上都有非随机因素影响（目前人类也无法实现真正的随机），那么假设彩票每次出的号码目标是让中奖人数最少，从而最大化利益，那么就是通过上个序列预测概率最小的下个序列；
    2. 全国彩民买彩票都会受前几期影响，主要是前一期，这里为了简化也只考虑前一期，然后分析这一期会出的号码，我们的模型就是反向预测。

In [15]:
# reshape X to be [samples, time steps, features]
hislen = history.shape[0]

history0 = history-1 #从0计数
dataX = np.array(history0.iloc[:hislen-1,:6])
x_oh = []
for i in dataX:
    x_oh.append(keras.utils.np_utils.to_categorical(i, 33))
x_oh = np.array(x_oh)
print(x_oh.shape)

dataY = np.array(history0.iloc[1:hislen,:6])
y_oh = []
for i in dataY:
    y_oh.append(keras.utils.np_utils.to_categorical(i, 33))

y_oh = np.array(y_oh)
print(y_oh.shape)
#X = np.reshape(dataX, (len(dataX), 7, 1))


(2277, 6, 33)
(2277, 6, 33)


In [20]:
def build_model(input_size, max_out_seq_len, hidden_size):
    model = Sequential()
    model.add(LSTM(input_dim=input_size, output_dim=hidden_size, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(hidden_size, activation="relu"))
    model.add(RepeatVector(max_out_seq_len))
    model.add(LSTM(hidden_size, return_sequences=True))
    model.add(TimeDistributed(Dense(output_dim=input_size, activation="linear")))
    model.compile(loss="mse", optimizer='adam')
    #print(model.summary())
    return model

model = build_model(33,6,256)

/Users/fire/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/fire/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=False, input_shape=(None, 33), units=256)`
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/fire/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=33)`
  


In [21]:
model.fit(x_oh,y_oh,
          batch_size=64,
          epochs=120,
          validation_split=0.1)

Train on 2049 samples, validate on 228 samples
Epoch 1/120
2049/2049 [==============================] - 4s - loss: 0.0292 - val_loss: 0.0289
Epoch 2/120
2049/2049 [==============================] - 3s - loss: 0.0286 - val_loss: 0.0288
Epoch 3/120
2049/2049 [==============================] - 2s - loss: 0.0284 - val_loss: 0.0288
Epoch 4/120
2049/2049 [==============================] - 3s - loss: 0.0283 - val_loss: 0.0288
Epoch 5/120
2049/2049 [==============================] - 2s - loss: 0.0282 - val_loss: 0.0287
Epoch 6/120
2049/2049 [==============================] - 2s - loss: 0.0282 - val_loss: 0.0287
Epoch 7/120
2049/2049 [==============================] - 3s - loss: 0.0281 - val_loss: 0.0288
Epoch 8/120
2049/2049 [==============================] - 2s - loss: 0.0281 - val_loss: 0.0287
Epoch 9/120
2049/2049 [==============================] - 2s - loss: 0.0280 - val_loss: 0.0287
Epoch 10/120
2049/2049 [==============================] - 2s - loss: 0.0280 - val_loss: 0.0288
Epoch 11/120

2049/2049 [==============================] - 3s - loss: 0.0229 - val_loss: 0.0332
Epoch 87/120
2049/2049 [==============================] - 3s - loss: 0.0226 - val_loss: 0.0337
Epoch 88/120
2049/2049 [==============================] - 3s - loss: 0.0229 - val_loss: 0.0332
Epoch 89/120
2049/2049 [==============================] - 3s - loss: 0.0222 - val_loss: 0.0334
Epoch 90/120
2049/2049 [==============================] - 3s - loss: 0.0218 - val_loss: 0.0335
Epoch 91/120
2049/2049 [==============================] - 3s - loss: 0.0221 - val_loss: 0.0336
Epoch 92/120
2049/2049 [==============================] - 3s - loss: 0.0218 - val_loss: 0.0338
Epoch 93/120
2049/2049 [==============================] - 3s - loss: 0.0215 - val_loss: 0.0340
Epoch 94/120
2049/2049 [==============================] - 3s - loss: 0.0216 - val_loss: 0.0336
Epoch 95/120
2049/2049 [==============================] - 3s - loss: 0.0213 - val_loss: 0.0343
Epoch 96/120
2049/2049 [==============================] - 3s - 

In [22]:
test_x = np.reshape(y_oh[-1], (1, 6, 33))
print('Last red ball: ',[np.argmax(x)+1 for x in test_x[0]])
test_y = model.predict(test_x)
final_red = [np.argmax(x)+1 for x in test_y[-1]]
#解决重号的问题 
count33 = 2
inWhile = 0
while len(set(final_red)) <6:
    inWhile = 1
    for i in range(1,6):
        if final_red[i] == final_red[i-1]:
            final_red[i]+=1
            if final_red[i]>33:
                final_red[i] -= count33
                final_red = sorted(final_red)
                count33+=1
print('Next red ball: ',final_red ,' \n是否修改过： ', '是' if inWhile else '否')

Last red ball:  [11, 16, 19, 22, 25, 30]
Next red ball:  [3, 4, 19, 22, 29, 32]  
是否修改过：  否


## 2.蓝球lstm 多到1
* 思路：蓝球单独处理，利用前N期（比如10）期号码做成一个时间序列，预测下一期的号码。即多到一的预测。

In [23]:
blueAll = np.array(history0.iloc[:,6:7])
seq_N = 20
blue_train = []
blue_label = []
for i in range(seq_N,len(blueAll)):
    x_tmp = blueAll[i-seq_N:i][::-1] ## 小trick。逆序
    x_tmp = keras.utils.np_utils.to_categorical(x_tmp, 16)
    blue_train.append(x_tmp)
    
    y_tmp = blueAll[i]
    y_tmp = keras.utils.np_utils.to_categorical(y_tmp, 16)
    blue_label.append(y_tmp)

blue_train = np.array(blue_train)
blue_label = np.array(blue_label)
blue_label = np.reshape(blue_label,(blue_label.shape[0],blue_label.shape[2]))
print(blue_train.shape)
print(blue_label.shape)

(2258, 20, 16)
(2258, 16)


In [24]:
def build_model2(input_size, seq_len, hidden_siz):
    model = Sequential()
    model.add(LSTM(hidden_siz, input_shape=( seq_len,input_size)))
    model.add(Dropout(0.1))
    model.add(Dense(input_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    #print(model.summary())
    return model

model2 = build_model2(16,seq_N,128)

In [25]:
model2.fit(blue_train,blue_label,
          batch_size=64,
          epochs=120,
          validation_split=0.1)

Train on 2032 samples, validate on 226 samples
Epoch 1/120
2032/2032 [==============================] - 2s - loss: 2.7737 - val_loss: 2.7717
Epoch 2/120
2032/2032 [==============================] - 1s - loss: 2.7656 - val_loss: 2.7733
Epoch 3/120
2032/2032 [==============================] - 1s - loss: 2.7591 - val_loss: 2.7707
Epoch 4/120
2032/2032 [==============================] - 1s - loss: 2.7531 - val_loss: 2.7804
Epoch 5/120
2032/2032 [==============================] - 2s - loss: 2.7436 - val_loss: 2.7825
Epoch 6/120
2032/2032 [==============================] - 2s - loss: 2.7333 - val_loss: 2.7775
Epoch 7/120
2032/2032 [==============================] - 2s - loss: 2.7207 - val_loss: 2.8013
Epoch 8/120
2032/2032 [==============================] - 1s - loss: 2.7105 - val_loss: 2.8034
Epoch 9/120
2032/2032 [==============================] - 2s - loss: 2.6932 - val_loss: 2.8579
Epoch 10/120
2032/2032 [==============================] - 1s - loss: 2.6809 - val_loss: 2.8798
Epoch 11/120

2032/2032 [==============================] - 1s - loss: 0.1755 - val_loss: 7.1755
Epoch 87/120
2032/2032 [==============================] - 1s - loss: 0.1165 - val_loss: 7.1116
Epoch 88/120
2032/2032 [==============================] - 1s - loss: 0.1041 - val_loss: 7.2364
Epoch 89/120
2032/2032 [==============================] - 1s - loss: 0.0670 - val_loss: 7.4001
Epoch 90/120
2032/2032 [==============================] - 1s - loss: 0.0434 - val_loss: 7.3801
Epoch 91/120
2032/2032 [==============================] - 1s - loss: 0.0339 - val_loss: 7.4696
Epoch 92/120
2032/2032 [==============================] - 1s - loss: 0.0292 - val_loss: 7.5367
Epoch 93/120
2032/2032 [==============================] - 1s - loss: 0.0276 - val_loss: 7.6039
Epoch 94/120
2032/2032 [==============================] - 1s - loss: 0.0237 - val_loss: 7.6481
Epoch 95/120
2032/2032 [==============================] - 1s - loss: 0.0252 - val_loss: 7.6771
Epoch 96/120
2032/2032 [==============================] - 1s - 

In [26]:
true_last =  keras.utils.np_utils.to_categorical(blueAll[-seq_N:], 16)
test_x1 = np.reshape(true_last, (1, 20, 16))
print('Last blue ball: ',[np.argmax(x)+1 for x in test_x1[0]])
test_y1 = model2.predict(test_x1)
final_blue = np.argmax(test_y1)
print('Next blue ball: ',final_blue+1)
print('Final all ball: ',final_red,final_blue+1)

Last blue ball:  [6, 16, 7, 1, 13, 7, 11, 10, 7, 11, 12, 5, 1, 10, 5, 1, 12, 10, 3, 8]
Next blue ball:  1
Final all ball:  [3, 4, 19, 22, 29, 32] 1
